In [1]:
## Follow the Guide to set up the connection
## https://cloud.google.com/bigquery/docs/visualize-jupyter

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../MelbDatathon2018/Auth/engine_Republic_of_Data_2018-afb04b72bd54.json"

%matplotlib inline

In [2]:
%load_ext google.cloud.bigquery

/Applications/anaconda/envs/py3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Connect with Pandas

In [3]:
import pandas as pd
print(pd.__version__)
pd.options.display.max_rows = 999

0.23.3


The below query will return results to tell which route each station belongs to. The query looks at all scan data of MODE = 2 for train mode. 64401 - 64408 are train stations that are missing from StopLocations.txt.

64405 and 64407 are only appearing in one train route respectively. Referring to PTV Route Map https://static.ptv.vic.gov.au/mapupload/Route/1488166014/9-M_TrainLinearMap_Lilydale_99x235_February2017.pdf


we can infer that 64405 is Richmond, and 64407 is Jolimont. 

The other stations starts with 644XX are City Loop stations and North Melbourne.

Looking at the result of the query, stations that are shared by multiple train lines will only appear once in the route with smaller route ids. For example, route 1 is Werribee line and stations in the City Loop are represented by stations of 644XX. All other train lines that share city loop do not have 644XX listed (except 64405 and 64407 in Lilydale line and South Morang line, but these two stations are not city loop stations). 

Another example is route 10 and 11. Route 10 is Lilydale line and Route 11 is Belgrave line. The two lines share stations from Ringwood all the way to the City. Therefore Route 10 lists all stations from Lilydale to Richmond 64405 (stations in City Loop are not listed as they are shared with Route 1), whereas Route 11 only lists stations from Belgrave to Heathmont, which is the last station before Ringwood. 

This also tells us that the count ScanOn/Off of Ringwood and all other shared stations is counting both Route 10 and Route 11. 

So we can further infer that count of ScanOn/Off of City Loop stations in Route 1 are actually counting all lines that running via the City Loop. 

This means that if we are going to find out the most popular train lines, we cannot only look by the count of ScanOn/Off.

In [4]:

%%bigquery df_count
select Mode,
       ParentRoute,
       RouteID ,
       ScanType,
       StopID,
       StopNameShort,
       StopNameLong,
       StopType,
       count(*)
  from clean.scans
  left join `data.stop_locations` 
    on StopID = StopLocationID
 where mode = 2   
group by Mode,
         ParentRoute,
         RouteID,
         ScanType,
         StopID,
         StopNameShort,
         StopNameLong,
         StopType
order by mode,
         ParentRoute,
         RouteID,
         ScanType,
         stopid,
         StopNameShort,
         StopNameLong,
         StopType

,Mode,ParentRoute,RouteID,ScanType,StopID,StopNameShort,StopNameLong,StopType,f0_
0,2,None,1,False,19921,Werribee,Werribee Railway Station (Werribee),Platform,456533
1,2,None,1,False,19922,Hoppers Crossing,Hoppers Crossing Railway Station (Hoppers Cros...,Platform,453745
2,2,None,1,False,19923,Laverton,Laverton Railway Station (Laverton),Platform,483571
3,2,None,1,False,19924,Aircraft,Aircraft Railway Station (Laverton),Platform,132856
4,2,None,1,False,19925,Westona,Westona Railway Station (Altona),Platform,93393
5,2,None,1,False,19926,Altona,Altona Railway Station (Altona),Platform,130186
6,2,None,1,False,19927,Seaholme,Seaholme Railway Station (Seaholme),Platform,42784
7,2,None,1,False,19994,Newport,Newport Railway Station (Newport),Platform,518610
8,2,None,1,False,19995,Spotswood,Spotswood Railway Station (Spotswood),Platform,136531
9,2,None,1,False,19996,Yarraville,Yarraville Railway Station (Yarraville),Platform,405282


Here are some examples I used to determine these stops.

- 64401 FLAGSTAFF    
- 64402 MELBOURNE CENTRAL CARDID 153300   
- 64403 PARLIAMENT CARDID 1865190   
- 64404 FLINDERS CARDID 61301   
- 64405 RICHMOND ROUTEID 10 LILYDALE LINE   
- 64406 NORTH MELBOURNE CARDID 100131   
- 64407 JOLIMONT ROUTEID 18 SOUTH MORANG LINE   
- 64408 SOUTHERN CROSS CARDID 126800   

I have created 2 CSV files:
- `raw_train_missing_stop_locations.csv`: listing all missing stops and their counts in unioned Sample_0 and Sample_1, with the StopID and StopLocationID mapped. Stations for regional train lines included.
- `train_missing_stop_locations.csv`: formatted exactly the same as Stop_locations.txt with the 644XX as the StopLocationID. Stations for regional train lines excluded. This should be imported to BigQuery.

You will find two rows in Stop_Locations.txt for each of Flinders Station, Southern Cross Station, Richmond and North Melbourne. e.g. for Flinders Station, there are stoplocationid 22238 and 19854, and if you check ptv data:   
-https://www.ptv.vic.gov.au/stop/view/22238    
-https://www.ptv.vic.gov.au/stop/view/19854    
you will find 22238 is for regional trains, and 19854 is for metro trains.

When I checked the train line details (mode 2), there is only one route (routeid 2) has the parentroute as 'Vline Rail'. But it seems to be Williamstown line. Therefore I assume there are no regional train line's scan activities included in the samples. So in `train_missing_stop_locations.csv` I excluded the regional train line stations.


